# **Automated Location Analysis Tool**


In [ ]:
import requests

## Overview 
The goal of the project is to create a dashboard, which outputs an automated location analysis for a specified address. The tool should be designed such that it is useful for a variety of actors, for whom a structured assessment of a location presents beneficial (productivity improvement). For instance, it could be useful for: 
- the general public: e.g. holiday makers assessing hotels or someone who wants to assess a potential new flat
- real estate agents researching attractive features of some property to publish in adverts
- office managers assessing potential office locations or researching amenities, e.g. to provide some suggestion to employees
- architects or city planners assessing the surrounding area of a site to inform decisions regarding potential uses
- corporates assessing possible locations for

Overall, depending on the area of application, the requirements regarding outputs for such a tool/dashboard might differ, as are the relative importance of features considered. Such could, for instance, be a representative/central location, rental prices, public transport connectivity, and the location's surrounding area with all amenities that it has to offer. 

In the most basic version, the dashboard will showcase an objective assessment of the latter two features. Especially considering the surrounding area, the relative importance of different categories of amenities will differ depending on user groups. Thus, aside from inputting the location (in the format (street, no., city, (country))), the user should choose a selection of amenities in order to allow for some individualization. 

The output will be in the form of a sorted list/table including the identified feature and its distance as the crow flies and via road to the location. 

If time permits, measures will be created that give some insight into the performance of a location regarding the other features. Also, reference locations could be generated to provide some measure of relative performance.





## Project Management:

### TP 0 - Structure
- Goals: 
    - create necessary directory structure to facilitate streamlined execution of the program
    - split tasks into various task points and identify points that can be worked on in parallel such as to optimise the time until completion of the basic model

### TP 1 - Input Processing (geopy)
- Goal: take a raw input by the user and transform it into the necessary format needed for subsequent analyses

- UI: text input - street, number, city, country (transformed into lon/lat coordinates using geopy) 
    - at this stage no output shown, will be passed silently to next task 
    - store adress in nice formatted way 


#### Geopy and Nominatim


In [22]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='automated_location_analysis')
location1 = geolocator.geocode('Carnotstr, 4, Berlin')

In [25]:
addressdetails = 0
q = ''
street = 'carnot straße, 4'
city = 'berlin'

nominatim_url = 'https://nominatim.openstreetmap.org/search'

params = {'addressdetails': addressdetails,
          'q':q,
          'street': street,
          'city': city,
          'format': 'json'}

response2 = requests.get(nominatim_url, params=params)

data2 = response2.json()

### TP 2 - Overpass API (overpy)
- Goal: query the overpass API using overpy to get a selection of different amenities within a specified radius around the location

- Subtasks:
    - figure out how to use the long/lat output from TP 1 to specify an **area to be queried**
    - collect different **types of amenities** from the overpass language guide/documentation (node types)
    - construct query
        - output json
        - parse names of amenities into list with lat/lon coordinates

    
- UI: suggestion - tick boxes to select amenities
    - output in background: df with the name, category, address, lat/lon
    - should check computing time for different settings, likely only a handful feasible, especially in really central locations
    - especially for the routing, limited capacity; unless install on local machine
    


#### Overpass request

Structure:
```python 
[out:json][timeout:25];
(
  node["amenity"="post_box"]({{bbox}});
  way["amenity"="post_box"]({{bbox}});
  relation["amenity"="post_box"]({{bbox}});
);
out body;
>;
out skel qt;
```

To define the query: 
- need to collect the nodes, ways, relations we want to collect
- define the bbox 
- out statement (e.g. out center to get single coordinate per object, likely preferable)

In [ ]:
overpass_url = 'https://z.overpass-api.de/api/interpreter'

overpass_query = """
[out:json];
area["ISO3166-1"="DE"][admin_level=2];
(node["amenity"="biergarten"](area);
 way["amenity"="biergarten"](area);
 rel["amenity"="biergarten"](area);
);
out center;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})

data = response.json()

In [ ]:
import overpy
api = overpy.Overpass()

query = """
[out:json];
area["ISO3166-1"="DE"][admin_level=2];
(node["amenity"="biergarten"](area);
 way["amenity"="biergarten"](area);
 rel["amenity"="biergarten"](area);
);
out center;"""

result = api.query(query)
data = result.json()

### TP 3 - Get distance
- Goal: use the lon/lat obtained in TP 1 and 
    1. compute the distance as the crow flies to the location of interest using geopy
    2. compute the distance via road using a to be selected API (e.g. openrouteservice)
    
- UI: output list/table/df; 
    - first location outputted in nice format
    - list/table/df with ammenities sorted by category (possibly split in multiple dfs)
        - columns: name, (category), address, distance as the crow flies, distance by road, approx time using modes of transport (simple interact with average speed factors for walking, bike, e-scooter)

#### Pyroutelib

In [ ]:
import openrouteservice
from openrouteservice import convert

coords = ((8.34234,48.23424),(8.34423,48.26424))

client = openrouteservice.Client(key='5b3ce3597851110001cf62482818c293528942238de6f690d9ec3b11') # Specify your personal API key

# decode_polyline needs the geometry only
geometry = client.directions(coords, profile='cycling-regular')['routes'][0]['geometry']

decoded = convert.decode_polyline(geometry)

print(decoded)

#### Openrouteservice

In [21]:
%%time
import requests
mode = 'foot-walking'
base_url = f'https://api.openrouteservice.org/v2/directions/{mode}'
body = {"coordinates":[[8.681495,49.41461],[8.686507,49.41943],[8.687872,49.420318]]}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf62482818c293528942238de6f690d9ec3b11',
    'Content-Type': 'application/json; charset=utf-8'
}

result = requests.post(base_url, json=body,headers=headers)

CPU times: user 27 ms, sys: 4.26 ms, total: 31.3 ms
Wall time: 187 ms


In [ ]:
result.json()

### TP 4 - (Optional) Multi-Criteria Decision Analysis
1. generate reference locations or loop over some list of scraped office locations in berlin and generate reference score or distribution (alternatively also create raster and compute for all boxes some score)
2. centrality of the location: generate random location within bounding box and get distance to inputted location

# References

- [Immobilienscout24](https://www.immobilienscout24.de/gewerbe/ratgeber/standortanalyse/standortkriterien-buero.html)